# Instructions:
1) In 'Specify Input/ Output Directories' code block, update: INPUT_PATH_FOLDER and OUTPUT_PATH_FOLDER
2) Update Input/Output Diretcories:
* If on Windows, make sure you have WSL installed. Then update WSL_INPUT_PATH_FOLDER and WSL_OUTPUT_PATH_FOLDER.
#### Important:  
#####    Make sure 'INPUT_PATH_FOLDER' is equivalent to 'WSL_INPUT_PATH_FOLDER' and 'OUTPUT_PATH_FOLDER' is equivalent to 'WSL_OUTPUT_PATH_FOLDER'
* If on Linux comment out windows code block, and uncomment out linux code block
3) Run all cells in notebook

If you are having issue (or are working on this code) see the developer notes section at the end

## Import Libraries/ Functions:

In [2]:
#general library:
import os
import shutil

#add sextractor to path
import sys
sys.path.insert(0, '../gibbi/sextractor')

#now we can import the run_sextractor method:
from run_sextractor import run_sextractor

## Specify Input/ Output Directories:

If on Windows, you need WSL (as Sextractor is run with WSL)

In [3]:
################################  IMPORTANT ################################:
#modify these paths:

#path to folder (as it appears on your native OS)
INPUT_PATH_FOLDER = '..\\..\\spin-parity-catalog\\galaxies\\table2'
OUTPUT_PATH_FOLDER = '..\\..\\spin-parity-catalog\\output_from_running\\sextractor\\table2'

#If on Windows, uncomment out this block (and comment out Linux block):
WSL_INPUT_PATH_FOLDER = '/mnt/c/Users/cora/Desktop/github/spin-parity-catalog/galaxies/table2' #for Windows Only
WSL_OUTPUT_PATH_FOLDER = '/mnt/c/Users/cora/Desktop/github/spin-parity-catalog/output_from_running/sextractor/table2' #for Windows Only

#If on LINUX, uncomment out this block (and comment out Windows block):
#WSL_INPUT_PATH_FOLDER = INPUT_PATH_FOLDER #for Linux Only
#WSL_OUTPUT_PATH_FOLDER = OUTPUT_PATH_FOLDER #for Linux Only
#########################################################################

In [4]:
#check input directory:
if not os.path.isdir(INPUT_PATH_FOLDER):
    print('INPUT_PATH_FOLDER = {}: does not exist, double check the INPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(INPUT_PATH_FOLDER))
   
#check if output directory exists, and if not make it:
if os.path.isdir(OUTPUT_PATH_FOLDER):
    pass
elif OUTPUT_PATH_FOLDER != '':
    os.makedirs(OUTPUT_PATH_FOLDER)
else:
    print('OUTPUT_PATH_FOLDER = {}: is blank, double check the OUTPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(INPUT_PATH_FOLDER))

## run sextractor on all fits files in nested INPUT_PATH_FOLDER:

In [5]:
def get_fits_files_in_folder(folder_path):
    fits_files = []
    for file in os.listdir(folder_path):
        if file.endswith(".fits") and "mask" not in file:
            fits_files.append(file)
    return fits_files

In [6]:
def run_on_fits_with_space_in_name(gal_name,fits_file,sex_path,input_folder,output_path):
    #Step 1: Make tmp directory
    os.makedirs(os.path.join(input_folder,'tmp')) #it will break if 'tmp' exists in input_folder
    
    #Step 2: Copy the file (but remove spaces in name)
    src = os.path.join(input_folder,gal_name,fits_file)
    dst = os.path.join(input_folder,'tmp','tmp.fits')
    shutil.copyfile(src, dst)
    
    #Step 3: create input/output path:
    input_sextractor_path = os.path.join(WSL_INPUT_PATH_FOLDER,'tmp','tmp.fits').replace("\\","/") #replace is to change to Linux style path (for Windows run)
    output_sextractor_path = os.path.join(WSL_INPUT_PATH_FOLDER,'tmp','tmp.sex').replace("\\","/") #replace is to change to Linux style path (for Windows run)
    
    #Step 4: run sextractor and move file to correct location (do some tidying up too)
    run_sextractor(input_sextractor_path,output_sextractor_path)
    src =  os.path.join(input_folder,'tmp','tmp.sex')
    dst = os.path.join(output_path,gal_name,sex_path)
    shutil.copyfile(src, dst)
    shutil.rmtree(os.path.join(input_folder,'tmp'))

In [7]:
def run_sextractor_with_all_fits(input_folder,output_path):
    for gal_name in os.listdir(input_folder):
        
        #Step 1: Create Output dir for specific galaxy
        print('Running on {}'.format(gal_name))
        gal_output_path = os.path.join(output_path,gal_name)
        os.makedirs(gal_output_path,exist_ok=True) #requires python 3.2+
        
        #Step 2: Get all fits files for galaxy
        fits_for_gal = get_fits_files_in_folder(os.path.join(input_folder,gal_name))
        
        #Step 3: Iterate through fits and run_sextractor on all fits
        for fits_file in fits_for_gal:
            #Step 3a: Get input_sextractor_path and output_sextractor_path path
            # i.e:
            #     for fits file 'IC1683/IC1683_g.fits' with OUTPUT_PATH_FOLDER = 'sextractor'
            #     input_sextractor_path = 'IC1683/IC1683_g.fits'
            #     output_sextractor_path = 'sextractor/IC1683/IC1683_g.sex'
            
            sex_path = '{}.sex'.format(os.path.splitext(fits_file)[0])

            #Handle case where galaxy has space in name:
            if ' ' in gal_name or ' ' in fits_file:
                run_on_fits_with_space_in_name(gal_name,fits_file,sex_path,input_folder,output_path)
            else:
                input_sextractor_path = os.path.join(WSL_INPUT_PATH_FOLDER,gal_name,fits_file).replace("\\","/") #replace is to change to Linux style path (for Windows run)
                output_sextractor_path = os.path.join(WSL_OUTPUT_PATH_FOLDER,gal_name,sex_path).replace("\\","/") #replace is to change to Linux style path (for Windows run)
            
                #Step 3b: run sextractor
                run_sextractor(input_sextractor_path,output_sextractor_path)

In [8]:
run_sextractor_with_all_fits(INPUT_PATH_FOLDER,OUTPUT_PATH_FOLDER)

Running on IC 2101
Running on IC1683
Running on IC1755
Running on IC2101
Running on IC5376
Running on MCG-02-02-030
Running on MCG-02-51-004
Running on NGC1035
Running on NGC1056
Running on NGC1084
Running on NGC1093
Running on NGC1097
Running on NGC1421
Running on NGC157
Running on NGC1637
Running on NGC1667
Running on NGC169
Running on NGC1961
Running on NGC2280
Running on NGC2347
Running on NGC24
Running on NGC2403
Running on NGC2410
Running on NGC2613
Running on NGC2639
Running on NGC2683
Running on NGC2715
Running on NGC2742
Running on NGC2775
Running on NGC2782
Running on NGC2841
Running on NGC2903
Running on NGC3160
Running on NGC3175
Running on NGC3198
Running on NGC3227
Running on NGC3310
Running on NGC3312
Running on NGC3368
Running on NGC3521
Running on NGC3623
Running on NGC3627
Running on NGC3646
Running on NGC3672
Running on NGC3675
Running on NGC3718
Running on NGC3815
Running on NGC3900
Running on NGC3949
Running on NGC4062
Running on NGC4088
Running on NGC4258
Running 

In [14]:
#Check size of input and output (and figure out if there was an issue running on something)
def compare_input_and_output(input_folder,output_path):
    input_count = 0
    output_count = 0
    for gal_name in os.listdir(input_folder):
        for file in os.listdir(os.path.join(input_folder,gal_name)):
            if file.endswith(".fits") and "mask" not in file:
                input_count += 1
                
                sex_path = '{}.sex'.format(os.path.splitext(file)[0])
                output = os.path.join(output_path,gal_name,sex_path)
                if os.path.exists(output):
                    output_count += 1
                else:
                    print('Missing {}'.format(sex_path))
    print("Input: {} galaxies, Output: {} source extractor files".format(input_count,output_count))
    
compare_input_and_output(INPUT_PATH_FOLDER,OUTPUT_PATH_FOLDER)

Input: 675 galaxies, Output: 675 source extractor files


## Developer Notes:
Last Updated: 06/22/2022

Important:
* If running on Windows, this requires WSL. Sextractor requires linux, and to run it we use WSL and run_sextractor.py creates a process (via WSL) and runs it
* Make sure 'INPUT_PATH_FOLDER' is equivalent to 'WSL_INPUT_PATH_FOLDER' and 'OUTPUT_PATH_FOLDER' is equivalent to 'WSL_OUTPUT_PATH_FOLDER' or ELSE... it will break :'(

    
Known Bugs:
1) On windows "#rerunm [WinError 3]The system cannot find the path specified" exception - Try rerunning the whole script

Things to Look out for:
1) Make sure to keep: os.chdir(original_working_dir) in run_sextractor.py remains
In order to run sextractor, we need to switch directories so if this line is removed, we may have issues in this script

2)Becareful with galaxy names with spaces, sextractor breaks if name has space, so temporary (somewhat hacky, immidetidaly) work around is to rename file (hence run_on_fits_with_space_in_name function)
I am aware that you can escape the spaces 'i.e. https://stackoverflow.com/a/65548647/13544635 or https://stackoverflow.com/a/25655627/13544635 ' however, the issue seems to be with sextractor format not linux so we rename the file. Because of renamming, if there is a folder called 'tmp' inside 'INPUT_PATH_FOLDER' it will break